In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from datasets import *
import matplotlib.pyplot as plt
import cv2
from keras.datasets import mnist
import tensorflow as tf
import numpy as np
from random import randint
import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers.merge import Concatenate
from keras.layers import Input, Reshape, Dense, Conv2D, Dropout, \
    MaxPooling2D, Flatten, UpSampling2D, Multiply, Activation, AveragePooling2D, \
    Add, Subtract, Lambda
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, TensorBoard
from scipy.ndimage import gaussian_filter as blur
from keras.optimizers import Adam
from keras.utils import multi_gpu_model as mgpu
from keras.utils import np_utils
import os, sys

In [ ]:
from models.yprob import yprob

model = yprob(imsize=256)

print(model.model.count_params())
model.compile()
print('LR', K.eval(model.model.optimizer.lr))

In [ ]:
dset = Tissue()

In [ ]:
datagen = dset.gen()
print(type(datagen))

for ii in range(50):
    imgs, lbls = next(datagen)
print(type(imgs))
print(type(lbls))
print(imgs.shape)
print(imgs[0].shape, imgs[0].dtype, np.max(imgs[0]))
print(lbls.shape)
print(lbls[0])


In [ ]:
plt.figure(figsize=(14, 5))
for ii in range(5):
    plt.subplot(1, 5, ii+1)
    plt.gca().set_title('%s: %.2f~%.2f' % (lbls[ii], np.min(imgs[ii]), np.max(imgs[ii])))
    plt.imshow(imgs[ii, :, :, 0], vmin=0, vmax=1)
plt.show()
plt.close()

In [ ]:
bsize=32

LOGFOLDER = 'logs'
TAG = 'mnist'
try: shutil.rmtree('./%s/%s' % (LOGFOLDER, TAG)) 
except: pass
board_cb = TensorBoard('%s/%s' % (LOGFOLDER, TAG))

datagen = dset.gen(bsize=bsize)
evalgen = dset.gen(bsize=bsize, mode='test')
model.model.fit_generator(
    datagen,
    epochs=4,
    verbose=1,
    steps_per_epoch=dset.train_size // bsize,
#     steps_per_epoch=1,
    validation_data=evalgen,
    validation_steps=dset.test_size // bsize // 10 # just do 10th of val data
)


In [ ]:
# TODO: visualize mistaking tiles